In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler,LabelEncoder
from sklearn.linear_model import Ridge,LinearRegression, LassoCV, Lasso,Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Conectarse a la base de datos
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="312714",
  database="TFM"
)

# Crear un dataframe con los datos de la tabla
df = pd.read_sql("SELECT * FROM mi_tabla", con=mydb)

# Cerrar la conexión
mydb.close()

# Imprimir el dataframe
print(df.head())

                   Track_Name         Artist  Position  Streams  Visita_Media  \
0  Reggaetón Lento (Bailemos)           CNCO         1    19272       8553.69   
1                    Chantaje        Shakira         2    19270       7129.82   
2   Otra Vez (feat. J Balvin)  Zion & Lennox         3    15761      10260.90   
3                Vente Pa' Ca   Ricky Martin         4    14954       5767.20   
4                      Safari       J Balvin         5    14269       6277.28   

   Visitas_media_Artist        Date Region  
0               6050.86  2017-01-01     ec  
1               7311.42  2017-01-01     ec  
2               7376.97  2017-01-01     ec  
3               5767.20  2017-01-01     ec  
4               7650.11  2017-01-01     ec  


In [2]:
# Seleccionar filas que contienen NaN
df_with_nan = df[df.isna().any(axis=1)]

# Imprimir el DataFrame con las filas que contienen NaN
print(df_with_nan)

Empty DataFrame
Columns: [Track_Name, Artist, Position, Streams, Visita_Media, Visitas_media_Artist, Date, Region]
Index: []


In [6]:
#REGRESION LINEAL SIN Visita_Media NI Visitas_media_Artist y con reduccion de variables para usar dummies     
from sklearn.linear_model import Ridge,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Seleccionar las columnas que se utilizarán como características
X = df[['Track_Name','Position', 'Region']]
y = df['Streams'] 

# Codificar las variables categóricas
X = pd.get_dummies(X, columns=['Region'])
X = pd.get_dummies(X, columns=['Track_Name'])


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo y entrenarlo
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el coeficiente R2 del modelo
r2 = r2_score(y_test, y_pred)
print("Coeficiente R2:", r2)
# Calcular el MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

Coeficiente R2: 0.6357175266016749
MAE: 43987.41903968771
MSE: 19381160137.360355
RMSE: 139216.23517880504


In [3]:
#REGRESION LINEAL SIN Visitas_media_Artist        

# Seleccionar las columnas que se utilizarán como características
X = df[['Artist','Track_Name','Position', 'Region','Visita_Media']]
y = df['Streams']

# Codificar las variables categóricas
X = pd.get_dummies(X, columns=['Region'])
X = pd.get_dummies(X, columns=['Track_Name'])
X = pd.get_dummies(X, columns=['Artist'])


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo y entrenarlo
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el coeficiente R2 del modelo
r2 = r2_score(y_test, y_pred)
print("Coeficiente R2:", r2)
# Calcular el MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

Coeficiente R2: 0.8250765607475374
MAE: 25904.896717984546
MSE: 9306566841.668398
RMSE: 96470.54908970093


In [4]:
##REGRESION LINEAL MULTIPLE SIN Visitas_media_Artist   

import numpy as np
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm

# Definir las variables independientes y la dependiente
y = df['Streams'] # variable dependiente
X_cat = df[['Artist', 'Track_Name', 'Region']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

# Crear un modelo de regresión lineal múltiple y ajustar los datos
model = sm.OLS(y, X).fit()

# Imprimir los resultados del modelo
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Streams   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                 2.348e+06
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        17:42:12   Log-Likelihood:            -3.3520e+07
No. Observations:             2597931   AIC:                         6.704e+07
Df Residuals:                 2597925   BIC:                         6.704e+07
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          2.54e+04    212.887    119.310   

In [5]:
# Crear un diccionario para contar el número de veces que aparece cada canción
canciones = {}

# Iterar sobre las filas del DataFrame
for index, row in df.iterrows():
    # Obtener el nombre de la canción y actualizar el diccionario
    cancion = row['Track_Name']
    if cancion in canciones:
        canciones[cancion] += 1
    else:
        canciones[cancion] = 1
        
# Imprimir el resultado
for cancion, count in canciones.items():
    print(f'{cancion}: {count}')

Reggaetón Lento (Bailemos): 7277
Chantaje: 10447
Otra Vez (feat. J Balvin): 6791
Vente Pa' Ca: 6384
Safari: 6757
La Bicicleta: 5485
Ay Mi Dios: 5320
Andas En Mi Cabeza: 5670
Traicionera: 6139
Shaky Shaky: 3511
Vacaciones: 6026
Dile Que Tu Me Quieres: 5682
Let Me Love You: 14562
DUELE EL CORAZON: 3500
Chillax: 4781
Borro Cassette: 5059
One Dance: 12440
Closer: 15136
Starboy: 13221
Desde Esa Noche: 2658
Don't Wanna Know: 8699
La Gozadera: 2968
24K Magic: 10234
Me Llamas (feat. Maluma) - Remix: 6229
Cuatro Babys: 3928
My Way: 7010
Gyal You A Party Animal: 1448
Baila Conmigo: 854
Gyal You A Party Animal - Remix: 4205
We Don't Talk Anymore (feat. Selena Gomez): 11779
Por Fin Te Encontré: 2473
CAN'T STOP THE FEELING! (Original Song from DreamWorks Animation's "TROLLS"): 8651
Bailame Despacio: 4078
I Took A Pill In Ibiza - Seeb Remix: 9831
Hasta el Amanecer: 3397
Cold Water (feat. Justin Bieber & MØ): 9444
Bobo: 1760
Ya Me Enteré (feat. Nicky Jam) - Urban Version: 3852
In the Name of Love: 10

In [3]:
#CAMBIAR VISITA_MEDIA_ARTIST DE NUMERICA A CATEGORICA 

df['Visitas_media_Artist'] = pd.cut(df['Visitas_media_Artist'], bins=11)
df['Visitas_media_Artist'] = df['Visitas_media_Artist'].cat.rename_categories(['0', '1', '2', '3', '4','5', '6', '7', '8', '9','10'])
# Mostrar el resultado
print(df)

                         Track_Name              Artist  Position  Streams  \
0        Reggaetón Lento (Bailemos)                CNCO         1    19272   
1                          Chantaje             Shakira         2    19270   
2         Otra Vez (feat. J Balvin)       Zion & Lennox         3    15761   
3                      Vente Pa' Ca        Ricky Martin         4    14954   
4                            Safari            J Balvin         5    14269   
...                             ...                 ...       ...      ...   
2597926          Castle on the Hill          Ed Sheeran       194     2134   
2597927           Whatever It Takes     Imagine Dragons       195     2130   
2597928          More Than You Know  Axwell /\ Ingrosso       196     2126   
2597929                      Issues      Julia Michaels       197     2117   
2597930                  Let Her Go           Passenger       200     2088   

         Visita_Media Visitas_media_Artist        Date Region  

In [4]:
# Crear un diccionario para contar el número de veces que aparece cada canción
popularidad = {}

# Iterar sobre las filas del DataFrame
for index, row in df.iterrows():
    # Obtener el nombre de la canción y actualizar el diccionario
    popu = row['Visitas_media_Artist']
    if popu in popularidad:
        popularidad[popu] += 1
    else:
        popularidad[popu] = 1
        
# Imprimir el resultado
for popu, count in popularidad.items():
    print(f'{popu}: {count}')

0: 2479897
1: 39921
2: 24277
3: 17217
4: 12387
5: 14435
6: 6688
10: 825
9: 1133
7: 1027
8: 124


In [8]:
##REGRESION LINEAL CON Visitas_media_Artist COMO VARIABLE CATEGORICA

# Seleccionar las columnas que se utilizarán como características
# En este caso, Streams se usará como variable independiente (X)
# y Position se usará como variable dependiente (y)
X = df[['Artist','Track_Name','Position', 'Region','Visita_Media','Visitas_media_Artist']]
y = df['Streams']
# Codificar las variables categóricas
X = pd.get_dummies(X, columns=['Region'])
X = pd.get_dummies(X, columns=['Track_Name'])
X = pd.get_dummies(X, columns=['Artist'])
X = pd.get_dummies(X, columns=['Visitas_media_Artist'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo y entrenarlo
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el coeficiente R2 del modelo
r2 = r2_score(y_test, y_pred)
print("Coeficiente R2:", r2)
# Calcular el MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

Coeficiente R2: 0.8249073344424619
MAE: 25925.570732342287
MSE: 9315570300.131626
RMSE: 96517.20209440195


In [9]:
##REGRESION LINEAL MULTIPLE CON Visitas_media_Artist COMO VARIABLE CATEGORICA

# Definir las variables independientes y la dependiente
y = df['Streams'] # variable dependiente
X_cat = df[['Artist', 'Track_Name', 'Region','Visitas_media_Artist']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

# Crear un modelo de regresión lineal múltiple y ajustar los datos
model = sm.OLS(y, X).fit()

# Imprimir los resultados del modelo
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Streams   R-squared:                       0.819
Model:                            OLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                 1.959e+06
Date:                Mon, 17 Apr 2023   Prob (F-statistic):               0.00
Time:                        17:58:14   Log-Likelihood:            -3.3519e+07
No. Observations:             2597931   AIC:                         6.704e+07
Df Residuals:                 2597924   BIC:                         6.704e+07
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 2.641e+04 

In [5]:
#Regresion multiple polinomial

y = df['Streams']
X_cat = df[['Artist', 'Track_Name', 'Region','Visitas_media_Artist']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)


model = LinearRegression()
model.fit(X_train_poly, y_train)


y_pred = model.predict(X_test_poly)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Coeficiente R2:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)


Coeficiente R2: 0.9170600504580299
MAE: 17780.79351079379
MSE: 4412708711.626152
RMSE: 66428.22225248958


In [16]:
%%time
#Regresion multiple polinomial

y = df['Streams']
X_cat = df[['Artist', 'Track_Name', 'Region','Visitas_media_Artist']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)


model = LinearRegression()
model.fit(X_train_poly, y_train)


y_pred = model.predict(X_test_poly)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("Coeficiente R2:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)

Coeficiente R2: 0.9477991470033301
MAE: 15225.273853941471
MSE: 2777276331.186562
RMSE: 52699.870314703454
Wall time: 17.5 s


Si tuvieramos pocos datos, es posible que un modelo de alto grado sobreajuste los datos, lo que significa que se ajustará demasiado bien a los datos de entrenamiento pero no generalizará bien a los datos nuevos.

Pero en nuestro caso tenemos muchos datos, por lo que nos podemos permitir un modelo más complejo, con un polinomio de mayor grado, ya que es más probable que el modelo generalice bien. Sin embargo, a pesar de que forma incial nuestra conclusión sea esta pasaremos a realizar pruebas más adelante que sirva para respaldar o contranstar este hecho.

# Resultados con diferntes grados:


Coeficiente R2: 0.9170600504580299
MAE: 17780.79351079379
MSE: 4412708711.626152
RMSE: 66428.22225248958

## Con polinomio de grado 3:

Coeficiente R2: 0.9477991470033301
MAE: 15225.273853941471
MSE: 2777276331.186562
RMSE: 52699.870314703454

## Con polinomio de grado 4:

Coeficiente R2: 0.9501781927522926
MAE: 14142.386862607327
MSE: 2650702394.744851
RMSE: 51484.97251378164

## Con polinomio de grado 5:

Coeficiente R2: 0.9543480128328105
MAE: 11557.133938019228
MSE: 2428852713.176093
RMSE: 49283.39185949048

Revisar validación cruzada, Lasso o Ridge

In [11]:
#Regresion multiple polinomial con validacion cruzada

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

# Definir las columnas para las variables independientes y dependiente
y = df['Streams']
X_cat = df[['Artist', 'Track_Name', 'Region','Visitas_media_Artist']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

# Crear el objeto PolynomialFeatures
poly = PolynomialFeatures(degree=2)

# Transformar las variables independientes usando PolynomialFeatures
X_poly = poly.fit_transform(X)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)

# Crear el objeto LinearRegression
lr = LinearRegression()

# Aplicar validación cruzada
scores = cross_val_score(lr, X_poly, y, cv=5, scoring='r2')

# Imprimir los resultados de validación cruzada
print("Resultados de validación cruzada:", scores)
print("R2 medio de validación cruzada:", np.mean(scores))

# Entrenar el modelo
lr.fit(X_train, y_train)

# Obtener las predicciones
y_pred = lr.predict(X_test)

# Obtener el coeficiente R2, MAE, MSE y RMSE
r2 = lr.score(X_test, y_test)
mae = np.mean(np.abs(y_test - y_pred))
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

# Imprimir los resultados
print("Coeficiente R2:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)

Resultados de validación cruzada: [ 0.8323094   0.7641157   0.52332964  0.79131876 -1.27755627]
R2 medio de validación cruzada: 0.32670344832107956
Coeficiente R2: 0.9169819052119582
MAE: 17710.791846945383
MSE: 4346561386.347227
RMSE: 65928.45657489053


# Resultados con variable categorica Visitas_media_Artist:

## Polinomio grados 2 y partiendo en cv=5:


Resultados de validación cruzada: [ 0.8323094   0.7641157   0.52332964  0.79131876 -1.27755627]
R2 medio de validación cruzada: 0.32670344832107956
Coeficiente R2: 0.9169819052119582
MAE: 17710.791846945383
MSE: 4346561386.347227
RMSE: 65928.45657489053

## Polinomio grados 3 y partiendo en cv=5:

Resultados de validación cruzada: [ 0.71520597  0.7159995   0.53598947  0.63005924 -6.33335877]
R2 medio de validación cruzada: -0.7472209164058217
Coeficiente R2: 0.9339653343763334
MAE: 25350.098627357216
MSE: 3457363463.8688254
RMSE: 58799.34917895627


# Resultados sin variable categorica Visitas_media_Artist:

## Polinomio grados 2 y partiendo en cv=5:

Resultados de validación cruzada: [0.81681224 0.76708486 0.35998752 0.70809231 0.14139327]
R2 medio de validación cruzada: 0.5586740376540913
Coeficiente R2: 0.8989990109888706
MAE: 20198.748471639923
MSE: 5288088096.209762
RMSE: 72719.24158164578

## Polinomio grados 3 y partiendo en cv=5:

Resultados de validación cruzada: [ 0.8200294   0.72297646 -0.05153426  0.09465545 -4.4783102 ]
R2 medio de validación cruzada: -0.5784366286099514
Coeficiente R2: 0.8516288017408085
MAE: 61319.51358516559
MSE: 7768240440.18356
RMSE: 88137.62216093398


# Conclusiones 

En este caso, se observa que el R2 medio de la validación cruzada ha mejorado en comparación con el modelo de grado 3, aunque sigue siendo negativo para uno de los pliegues, lo que indica que el modelo aún tiene problemas de sobreajuste.

Por otro lado, el coeficiente R2 en el conjunto de prueba es alto, lo que sugiere que el modelo ajustado puede explicar una buena parte de la variación en la variable objetivo. Sin embargo, el valor de MAE es mayor que en el modelo de grado 3, lo que indica que el modelo puede estar cometiendo errores más grandes en la predicción de los valores reales. El valor de RMSE también es mayor que en el modelo de grado 3.

De esta manera, se podría decir que este modelo de grado 2 tiene un desempeño razonable, pero todavía hay margen para mejoras. Se podría probar con diferentes técnicas de selección de características, regularización de modelos, entre otras técnicas para mejorar el rendimiento del modelo.

In [6]:
# Regresion multiple polinomial con validacion cruzada CON LASSO y variables numericas normalizadas
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

# Definir las columnas para las variables independientes y dependiente
y = df['Streams']
X_cat = df[['Artist', 'Track_Name', 'Region','Visitas_media_Artist']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Normalizar las variables numéricas
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)
X = pd.concat([X_cat_encoded, X_num_scaled], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

# Crear el objeto PolynomialFeatures
poly = PolynomialFeatures(degree=2)

# Transformar las variables independientes usando PolynomialFeatures
X_poly = poly.fit_transform(X)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)

# Crear el objeto LinearRegression
lr = LinearRegression()

# Crear el objeto Lasso para realizar selección de características
lasso = Lasso(alpha=0.1)

# Aplicar validación cruzada
scores = cross_val_score(lasso, X_train, y_train, cv=5, scoring='r2')

# Imprimir los resultados de validación cruzada
print("Resultados de validación cruzada:", scores)
print("R2 medio de validación cruzada:", np.mean(scores))

# Entrenar el modelo
lr.fit(X_train, y_train)

# Obtener las predicciones
y_pred = lr.predict(X_test)

# Obtener el coeficiente R2, MAE, MSE y RMSE
r2 = lr.score(X_test, y_test)
mae = np.mean(np.abs(y_test - y_pred))
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

# Imprimir los resultados
print("Coeficiente R2:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)

Resultados de validación cruzada: [0.9150067  0.91607476 0.91835532 0.91166872 0.91127589]
R2 medio de validación cruzada: 0.9144762764944557
Coeficiente R2: 0.9169994135215097
MAE: 17707.146080432183
MSE: 4345644707.369886
RMSE: 65921.50413461366


In [11]:
# Regresion multiple polinomial con validacion cruzada CON LASSO y variables numericas normalizadas


# Definir las columnas para las variables independientes y dependiente
y = df['Streams']
X_cat = df[['Artist', 'Track_Name', 'Region','Visitas_media_Artist']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Normalizar las variables numéricas
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)
X = pd.concat([X_cat_encoded, X_num_scaled], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

# Crear el objeto PolynomialFeatures
poly = PolynomialFeatures(degree=4)

# Transformar las variables independientes usando PolynomialFeatures
X_poly = poly.fit_transform(X)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)

# Crear el objeto LinearRegression
lr = LinearRegression()

# Crear el objeto Lasso para realizar selección de características
lasso = Lasso(alpha=0.1)

# Aplicar validación cruzada
scores = cross_val_score(lasso, X_train, y_train, cv=5, scoring='r2')

# Imprimir los resultados de validación cruzada
print("Resultados de validación cruzada:", scores)
print("R2 medio de validación cruzada:", np.mean(scores))

# Entrenar el modelo
lr.fit(X_train, y_train)

# Obtener las predicciones
y_pred = lr.predict(X_test)

# Obtener el coeficiente R2, MAE, MSE y RMSE
r2 = lr.score(X_test, y_test)
mae = np.mean(np.abs(y_test - y_pred))
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

# Imprimir los resultados
print("Coeficiente R2:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)

Resultados de validación cruzada: [0.96740485 0.96632754 0.96957618 0.96347805 0.96330653]
R2 medio de validación cruzada: 0.9660186303470724
Coeficiente R2: 0.9705031669223788
MAE: 10904.031824139774
MSE: 1544359648.364161
RMSE: 39298.341547248034


# LASSO: 0,1

### Resultados con variable categorica Visita_Media_artist:

Resultados de validación cruzada: [0.9150067  0.91607476 0.91835532 0.91166872 0.91127589]
R2 medio de validación cruzada: 0.9144762764944557
Coeficiente R2: 0.9169994135215097
MAE: 17707.146080432183
MSE: 4345644707.369886
RMSE: 65921.50413461366

### Resultados sin la variable categorica Visita_Media_artist:

Resultados de validación cruzada: [0.89959032 0.90001499 0.9007685  0.89488836 0.89284265]
R2 medio de validación cruzada: 0.8976209650246272
Coeficiente R2: 0.8989990109890682
MAE: 20198.74847164828
MSE: 5288088096.19945
RMSE: 72719.24158157488

## Polinomio grados 3 y partiendo en cv=5:

Resultados de validación cruzada: [0.95512474 0.95327998 0.9564696  0.94976132 0.94905816]
R2 medio de validación cruzada: 0.9527387605614959
Coeficiente R2: 0.9544680134469168
MAE: 13411.527115403887
MSE: 2383908894.809083
RMSE: 48825.289500514824

## Polinomio grados 4 y partiendo en cv=5:

Resultados de validación cruzada: [0.96740485 0.96632754 0.96957618 0.96347805 0.96330653]
R2 medio de validación cruzada: 0.9660186303470724
Coeficiente R2: 0.9705031669223788
MAE: 10904.031824139774
MSE: 1544359648.364161
RMSE: 39298.341547248034

## Polinomio grados 3 y partiendo en cv=5 Lasso= 0,2:

Resultados de validación cruzada: [0.95512473 0.95328012 0.9564697  0.94976146 0.94905823]
R2 medio de validación cruzada: 0.952738847191682
Coeficiente R2: 0.9544680134469168
MAE: 13411.527115403887
MSE: 2383908894.809083
RMSE: 48825.289500514824

# Conclusiones

Los resultados muestran que el modelo construido con Lasso es superior al modelo de regresión lineal con polinomio de grado dos. El coeficiente R2 es muy similar en ambos modelos, pero el modelo Lasso tiene una media de validación cruzada más alta, lo que indica una mayor capacidad de generalización. Además, el error medio absoluto (MAE), el error cuadrático medio (MSE) y la raíz del error cuadrático medio (RMSE) son ligeramente menores en el modelo Lasso. Esto sugiere que el modelo Lasso tiene un mejor ajuste a los datos y puede predecir los valores de la variable de respuesta con mayor precisión.

### lasso 0,3:
Resultados de validación cruzada: [0.91500683 0.91607484 0.91835513 0.91166862 0.91127608]
R2 medio de validación cruzada: 0.9144763021306203
Coeficiente R2: 0.9169994135215097
MAE: 17707.146080432183
MSE: 4345644707.369886
RMSE: 65921.50413461366

### lasso 0,1:
Resultados de validación cruzada: [0.9150067  0.91607476 0.91835532 0.91166872 0.91127589]
R2 medio de validación cruzada: 0.9144762764944557
Coeficiente R2: 0.9169994135215097
MAE: 17707.146080432183
MSE: 4345644707.369886
RMSE: 65921.50413461366

## Despues de los ajustes: lasso = LassoCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5, max_iter=20000)
grado 2:
Coeficiente R2: 0.9169795471153833
MAE: 17697.053988226628
MSE: 4346684848.726704
RMSE: 65929.39290427831

grado 3:
Coeficiente R2: 0.9544755751828924
MAE: 13411.38943531905
MSE: 2383512986.5471354
RMSE: 48821.234996127816

# Prueba del siguiente cuadro de código:

Grado 2: 
Resultados de validación cruzada: [0.91500664 0.9160777  0.91834258 0.91166877 0.91127579]
R2 medio de validación cruzada: 0.9144742942987646
Coeficiente R2: 0.9169795471153833
MAE: 17697.053988226628
MSE: 4346684848.726704
RMSE: 65929.39290427831

In [26]:
%%time
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
import statsmodels.api as sm

# Definir las columnas para las variables independientes y dependiente
y = df['Streams']
X_cat = df[['Artist', 'Track_Name', 'Region', 'Visitas_media_Artist']] # variables categóricas
X_num = df[['Position', 'Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Normalizar las variables numéricas
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)
X = pd.concat([X_cat_encoded, X_num_scaled], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

# Crear el objeto PolynomialFeatures
poly = PolynomialFeatures(degree=2)

# Transformar las variables independientes usando PolynomialFeatures
X_poly = poly.fit_transform(X)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)

# Crear el objeto LassoCV para realizar selección de características con validación cruzada
lasso = LassoCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5, max_iter=20000)

# Entrenar el modelo con LassoCV
lasso.fit(X_train, y_train)

# Aplicar validación cruzada con LassoCV
scores = cross_val_score(lasso, X_train, y_train, cv=5, scoring='r2')

# Imprimir los resultados de validación cruzada
print("Resultados de validación cruzada:", scores)
print("R2 medio de validación cruzada:", np.mean(scores))



# Obtener las predicciones
y_pred = lasso.predict(X_test)

# Obtener el coeficiente R2, MAE, MSE y RMSE
r2 = lasso.score(X_test, y_test)
mae = np.mean(np.abs(y_test - y_pred))
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

# Imprimir loss
print("RMSE:", rmse)


Resultados de validación cruzada: [0.91500664 0.91607472 0.91835659 0.91167076 0.91127028]
R2 medio de validación cruzada: 0.9144757975322791
Coeficiente R2: 0.9169795471153833
MAE: 17697.053988226628
MSE: 4346684848.726704
RMSE: 65929.39290427831
Wall time: 1h 55min 14s


In [28]:
%%time
#Regresion multiple polinomial con validacion cruzada CON Ridge y variables numericas normalizadas


# Definir las columnas para las variables independientes y dependiente
y = df['Streams']
X_cat = df[['Artist', 'Track_Name', 'Region','Visitas_media_Artist']] # variables categóricas
X_num = df[['Position','Visita_Media']] # variables numéricas

# Codificar las variables categóricas con LabelEncoder
le = LabelEncoder()
X_cat_encoded = X_cat.apply(le.fit_transform)

# Concatenar las variables categóricas codificadas y las numéricas en un solo dataframe
X = pd.concat([X_cat_encoded, X_num], axis=1)

# Normalizar las variables numéricas
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)
X = pd.concat([X_cat_encoded, X_num_scaled], axis=1)

# Agregar una constante a la matriz de variables independientes
X = sm.add_constant(X)

# Crear el objeto PolynomialFeatures
poly = PolynomialFeatures(degree=2)

# Transformar las variables independientes usando PolynomialFeatures
X_poly = poly.fit_transform(X)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42) #shuffle=True

# Definir la grilla de hiperparámetros para Ridge
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} #(-5, 5, 11) (-3, 3, 7)  {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Crear el objeto Ridge
ridge = Ridge()

# Aplicar GridSearchCV
grid_search = GridSearchCV(ridge, param_grid=param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Imprimir los resultados de GridSearchCV
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Entrenar el modelo con los mejores hiperparámetros
ridge_best = Ridge(alpha=grid_search.best_params_['alpha'])
ridge_best.fit(X_train, y_train)

# Obtener las predicciones
y_pred = ridge_best.predict(X_test)

# Obtener el coeficiente R2, MAE, MSE y RMSE
r2 = ridge_best.score(X_test, y_test)
mae = np.mean(np.abs(y_test - y_pred))
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)

# Imprimir los resultados
print("Coeficiente R2:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)

Mejores hiperparámetros: {'alpha': 0.001}
Mejor puntuación de validación cruzada: 0.9144762635536485
Coeficiente R2: 0.916981905367182
MAE: 17710.791681940176
MSE: 4346561378.2203665
RMSE: 65928.4565132566
Wall time: 28.1 s
